### OS Setup

In [ ]:
! pip install langchain yt_dlp openai pydub pypdf python-dotenv

In [2]:
import os
import openai
import sys
from dotenv import load_dotenv, find_dotenv

# Mount your Google Drive
from google.colab import drive

# Mount Google Drive
drive.mount('/content/data')

# Specify the path to the .env file
dotenv_path = 'data/MyDrive/Notebooks/env/.env'

# Load environmental variables from .env
load_dotenv(dotenv_path)

# Set your OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

Mounted at /content/data


### **Video to Documnents**

In [3]:
from langchain.document_loaders.generic import GenericLoader

#OpenAIWhisperParser : It use Openai Whisper model, spech to text model
# to convert youtube audio to txt format
from langchain.document_loaders.parsers import OpenAIWhisperParser

#YoutubeAudioLoader: Load audio file from youtube video
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

In [4]:
# Toatl Cost : $0.19
url="https://www.youtube.com/watch?v=ed9346Z4uLk"
save_dir="youtube/"
loader = GenericLoader(
    YoutubeAudioLoader([url],save_dir),
    OpenAIWhisperParser()
)

docs = loader.load()

[youtube] Extracting URL: https://www.youtube.com/watch?v=ed9346Z4uLk
[youtube] ed9346Z4uLk: Downloading webpage
[youtube] ed9346Z4uLk: Downloading ios player API JSON
[youtube] ed9346Z4uLk: Downloading android player API JSON
[youtube] ed9346Z4uLk: Downloading m3u8 information
[info] ed9346Z4uLk: Downloading 1 format(s): 140
[download] Destination: youtube//100 Years of NFL History In Under 4 Minutes!.m4a
[download] 100% of    3.27MiB in 00:00:00 at 8.36MiB/s   
[FixupM4a] Correcting container of "youtube//100 Years of NFL History In Under 4 Minutes!.m4a"
[ExtractAudio] Not converting audio youtube//100 Years of NFL History In Under 4 Minutes!.m4a; file is already in target format m4a
Transcribing part 1!


In [5]:
docs[0].page_content[0:10]

'In 1920, t'

In [6]:
len(docs)

1

In [7]:
page = docs[0]

In [8]:
page.metadata

{'source': 'youtube/100 Years of NFL History In Under 4 Minutes!.m4a',
 'chunk': 0}

In [9]:
page.page_content[0:150]

'In 1920, the American Professional Football Association was born, becoming the NFL two years later. In 1933, the Chicago Bears defeated the New York G'

### **Document Splitting**

In [10]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [11]:
splits = text_splitter.split_documents(docs)

In [12]:
len(splits)

3

In [13]:
print(splits[0])


page_content="In 1920, the American Professional Football Association was born, becoming the NFL two years later. In 1933, the Chicago Bears defeated the New York Giants in the league's inaugural championship game. Though the league featured black players prior, in 1946, the NFL officially integrated. In the 1958 championship game, the Baltimore Colts defeated the New York Giants in sudden death in what became known as the greatest game ever played. 45 million people tuned in, starting an explosion in viewership for future decades. The NFL merged with the American Football League in 1966, with the two league champions playing each other in the first AFL-NFL championship game, which later became known as Super Bowl I. The 1967 championship game was also known as the Ice Bowl, played at negative 13 degrees, so cold it froze the commentators' coffee. This was the Green Bay Packers' fourth NFL title in six years. They then won their second straight Super Bowl title. By 1969, Kansas City Ch

In [14]:
print(splits[1])

page_content="in the Super Bowl era to finish undefeated. They beat the Redskins in Super Bowl VII to finish the season 17-0. In 1978, newly introduced rules made defending the pass more difficult. The Steelers capitalized, winning the next two Super Bowls, giving them four in six seasons, cementing a 1970s dynasty. Dwight Clark made the catch to take the 49ers to the 1982 Super Bowl, sparking their 80s dynasty in the era of the pass-heavy West Coast offense. The Cowboys were next in the 90s, winning Super Bowls in 93, 94, and 96. After the attacks of September 11th devastated America, many people found a semblance of normalcy in the 2001 NFL season. That year, a week-two injury to Drew Bledsoe of the Patriots allowed little-known second-year quarterback Tom Brady to take the NFL stage. That year, Brady and the Patriots won the first of six Super Bowls over an 18-season stretch. They entered the NFL's 100th season as defending champions, and Brady has the consensus choice as the greate

In [15]:
print(splits[2])

page_content="its 100th season. Twenty NFL teams have won at least one of the first 53 Super Bowls. During the season, NFL games are played three days a week and shown on TV networks and streaming services around the globe. The league's structure and revenue sharing amongst all teams have kept the NFL the most popular sports league in the U.S. for several decades and is poised to remain so for many more. Here's to the next 100 years." metadata={'source': 'youtube/100 Years of NFL History In Under 4 Minutes!.m4a', 'chunk': 0}


In [16]:
for doc in docs:
    print(doc.metadata)

{'source': 'youtube/100 Years of NFL History In Under 4 Minutes!.m4a', 'chunk': 0}


In [17]:
! pip install reportlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.5 MB/s eta 0:00:00


### **Embedding & Vector Store**

In [ ]:
! pip install langchain_openai
! pip install langchain[docarray]
! pip install chromadb

In [19]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
persist_directory = 'docs/chroma/'

In [20]:
! rm -rf $persist_directory

In [21]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [22]:
print(vectordb._collection.count())

3


In [23]:
from langchain.vectorstores import DocArrayInMemorySearch

# create vector database from data
db = DocArrayInMemorySearch.from_documents(
    docs,
    embeddings)

In [24]:
#Save this so we can use it later!
vectordb.persist()

In [32]:
question = "Write a summary in no more then 5 sentance."

result = vectordb.similarity_search(question,k=3)

len(result)

3

In [33]:
print(result)

[Document(page_content="its 100th season. Twenty NFL teams have won at least one of the first 53 Super Bowls. During the season, NFL games are played three days a week and shown on TV networks and streaming services around the globe. The league's structure and revenue sharing amongst all teams have kept the NFL the most popular sports league in the U.S. for several decades and is poised to remain so for many more. Here's to the next 100 years.", metadata={'chunk': 0, 'source': 'youtube/100 Years of NFL History In Under 4 Minutes!.m4a'}), Document(page_content="in the Super Bowl era to finish undefeated. They beat the Redskins in Super Bowl VII to finish the season 17-0. In 1978, newly introduced rules made defending the pass more difficult. The Steelers capitalized, winning the next two Super Bowls, giving them four in six seasons, cementing a 1970s dynasty. Dwight Clark made the catch to take the 49ers to the 1982 Super Bowl, sparking their 80s dynasty in the era of the pass-heavy Wes

### **QA with Data**

In [37]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [38]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model_name=llm_name,
    temperature=0   # Good to get Actual answers becuase it is going to have low variability and usually just five us the higehest fidelity, most reliable answers
    )

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [39]:
from langchain.prompts import PromptTemplate

#Build Prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [44]:
from langchain.chains import RetrievalQA

# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=False,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [45]:
question = "Which game are we talking about ?"

In [46]:
result = qa_chain({"query": question})

In [47]:
print(result)

{'query': 'Which game are we talking about ?', 'result': 'The game being referred to is the 1958 championship game between the Baltimore Colts and the New York Giants, also known as the greatest game ever played. Thanks for asking!'}


In [48]:
question = "What is Super Bowl ?"

result = qa_chain({"query": question})

print(result)

{'query': 'What is Super Bowl ?', 'result': "The Super Bowl is the championship game of the National Football League (NFL), played annually between the champions of the league's two conferences. It is the culmination of the NFL season and is one of the most-watched sporting events in the United States. Thanks for asking!"}


In [49]:
question = "How many team has won Super Bowl ?"

result = qa_chain({"query": question})

In [50]:
print(result)

{'query': 'How many team has won Super Bowl ?', 'result': 'Twenty NFL teams have won at least one of the first 53 Super Bowls. Thanks for asking!'}


In [51]:
question = "Which team rank number 1 in total number of Super Bowl ?"

result = qa_chain({"query": question})

In [52]:
print(result)

{'query': 'Which team rank number 1 in total number of Super Bowl ?', 'result': 'The Pittsburgh Steelers rank number 1 in total number of Super Bowl wins with six championships. Thanks for asking!'}
